# **Hilal Durmaz**
**Approach3:** Collaborative filtering combined with global baseline 

In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

In [2]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [3]:
#ıtem to ıtem collaborative filtering
U = data.pivot("movieID", "userID", "rating")
U.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_size = 500
train = U[:train_size]
test = U[train_size:]

In [5]:
train.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
514,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN
516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#to avoid data leakage
sum=0
movie_avg_test=0
for movie_ıd in test.index:
  movie=test.dropna(subset=[movie_ıd],axis=1)
  movie_mean=movie.loc[movie_ıd:movie_ıd, :].mean(axis=1).squeeze()
  sum+=movie_mean

movie_avg_test=sum / test.shape[0]
print("test datasında bulunan filmlerin ortalaması",movie_avg_test)

test datasında bulunan filmlerin ortalaması 3.248346467499014


In [8]:
sum=0
movie_avg_train=0
for movie_ıd in train.index:
  movie=train.dropna(subset=[movie_ıd],axis=1)
  movie_mean=movie.loc[movie_ıd:movie_ıd, :].mean(axis=1).squeeze()
  sum+=movie_mean

movie_avg_train=sum / train.shape[0]
print("train datasında bulunan filmlerin ortalaması",movie_avg_train)

train datasında bulunan filmlerin ortalaması 3.180186095036204


In [ ]:
k=10
myPreds = []
realLabels = []
movie_mean_list=[]
baseline_list=[]
fark_list=[]
for movie_ıd in test.index:
  print(movie_ıd)
  if movie_ıd == 520:
    break
  for user_ıd in test.columns:
    if not np.isnan(test.loc[movie_ıd, user_ıd]):
      filtered_train = train.dropna(subset=[user_ıd])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)

    
      if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        test_row = test.loc[movie_ıd:movie_ıd, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0][0] #As a result of kneighbors, it comes as a list of the list and the first list is distance and the second list indexes
        #to calculate user bias 
        user = test.dropna(subset=[user_ıd])
        user_bias=movie_avg_test- user[user_ıd].mean(axis=0)
        #to calculate movie bias 
        movie = test.dropna(subset=[movie_ıd],axis=1)
        movie_bias=movie_avg_test-movie.loc[movie_ıd:movie_ıd, :].mean(axis=1).squeeze()

        baseline_userıd=movie_avg_test+user_bias+ movie_bias

        movie_idx = filtered_train.iloc[indices, :].index
        #I added the values the user gives to the closest movies to a list.
        rate=train.loc[movie_idx, user_ıd].values

        user_train = train.dropna(subset=[user_ıd])
        user_avg_train=user_train[user_ıd].mean(axis=0)

        movie_mean_list=[]
        for idx in movie_idx:
          movie = train.dropna(subset=[idx],axis=1)
          moviem_mean=movie.loc[idx:idx, :].mean(axis=1).squeeze()
          movie_mean_list.append(moviem_mean)
#I calculated the baseline by calculating the user bias and movie bias for each neighboring movie.
        fark_list=[]
        for i in range(len(movie_mean_list)):
          user_bias=user_avg_train-movie_avg_train
          movie_bias=movie_mean_list[i]-movie_avg_train
          baseline=movie_avg_train +user_bias+movie_bias
          baseline_list.append(baseline)
          fark=rate[i]-baseline  #I extracted the baseline from the users movie rating.
          fark_list.append(fark)  
        
        toplam=0
        toplam_distance=0
        for i in range(len(fark_list)):
          toplam += fark_list[i]*distances[i] #I defined the proximity of the films i and j as the distance
          toplam_distance +=distances[i]
        
        rating= baseline_userıd +(toplam/ toplam_distance)
        print(rating)
        myPreds.append(rating)
          

        real_label = test.loc[movie_ıd, user_ıd]
        realLabels.append(real_label)

In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.867519026585721

# Prediction

In [10]:
test= pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [11]:
x=list(test.IDs.values)
userid=[]
movieId=[]

x=list(test.IDs.values)
for i in range(test.shape[0]):
  user,movie=x[i].split("_")
  userid.append(user)
  movieId.append(movie)


d = {'UserId':userid , 'MovieId': movieId}
df_Test = pd.DataFrame(data=d)

In [12]:
df_Test.head()

,UserId,MovieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697


In [13]:
sum=0
movie_avg_test=0
for movie_ıd in U.index:
  movie=U.dropna(subset=[movie_ıd],axis=1)
  movie_mean=movie.loc[movie_ıd:movie_ıd, :].mean(axis=1).squeeze()
  sum+=movie_mean

movie_avg_test=sum / U.shape[0]
print("Bütün filmlerin ortalama",movie_avg_test)

Bütün filmlerin ortalama 3.23914803666732


In [ ]:
k = 10
myPreds = []
for i in range(len(df_Test)):
    userId=int(df_Test.UserId[i])
    movieId=int(df_Test.MovieId[i])
    filtered_train = U.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    
    if k < len(filtered_train):
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)
    
        test_row = U.loc[movieId:movieId, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0][0]

        user = U.dropna(subset=[userId])
        user_bias=movie_avg_test- user[userId].mean(axis=0)

        movie = U.dropna(subset=[movieId],axis=1)
        movie_bias=movie_avg_test-movie.loc[movieId:movieId, :].mean(axis=1).squeeze()

        baseline_userıd=movie_avg_test+user_bias+ movie_bias

        movie_idx = filtered_train.iloc[indices, :].index
        
        rate=U.loc[movie_idx, userId].values

        user_train = U.dropna(subset=[userId])
        user_avg_train=user_train[userId].mean(axis=0)

        movie_mean_list=[]
        for idx in movie_idx:
            movie = U.dropna(subset=[idx],axis=1)
            moviem_mean=movie.loc[idx:idx, :].mean(axis=1).squeeze()
            movie_mean_list.append(moviem_mean)

        fark_list=[]
        for i in range(len(movie_mean_list)):
            user_bias=user_avg_train-movie_avg_test
            movie_bias=movie_mean_list[i]-movie_avg_test
            baseline=movie_avg_test +user_bias+movie_bias
            baseline_list.append(baseline)
            fark=rate[i]-baseline
            fark_list.append(fark)  
        
        toplam=0
        toplam_distance=0
        for i in range(len(fark_list)):
            toplam += fark_list[i]*distances[i]
            toplam_distance +=distances[i]
        
        rating= baseline_userıd +(toplam/ toplam_distance)
        print(rating)
        myPreds.append(rating)
    
    else:
        user = U.dropna(subset=[userId])
        user_bias=movie_avg_test- user[userId].mean(axis=0)

        movie = U.dropna(subset=[movieId],axis=1)
        movie_bias=movie_avg_test-movie.loc[movieId:movieId, :].mean(axis=1).squeeze()

        baseline_userıd=movie_avg_test+user_bias+ movie_bias
        myPreds.append(baseline_userıd)

    print("kaç kayıt",len(myPreds))      


In [19]:
submission = pd.DataFrame({"IDs":list(test.IDs.values), "rating":myPreds})
submission.to_csv("submission2_apc3.csv", index = None)
from google.colab import files

files.download('submission2_apc3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**User to User - Global Baseline**

In [ ]:
myPreds = []
realLabels = []
k = 100
baseline=0
movie_bias=0
ıser_bias=0
toplam_distance=0
toplam2=0
userIds=0
for userId in test.index:
  print(userId)
  if userId == 502:
    break
  
  for movieId in test.columns:

    if not np.isnan(test.loc[userId, movieId]): 
      #I found the 100 users closest to the user by filtering by movieId
      filtered_train = train.dropna(subset=[movieId])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)
      
      if k>filtered_train.shape[0]:
        #I added the baseline value to my forecast list in case the filtered_train dimension is smaller than the k value.
        user = test.dropna(subset=[userId],axis=1)
        user_bias=user.loc[userId:userId, :].mean(axis=1).squeeze() - movie_avg
        movie = test.dropna(subset=[movieId])
        movie_bias=movie[movieId].mean()- movie_avg
        baseline=movie_avg+movie_bias+user_bias
        myPreds.append(baseline)
        real_label = U.loc[userId, movieId]
        realLabels.append(real_label)
        
      else:
        nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
        nn.fit(filtered_train)

        user = test.dropna(subset=[userId],axis=1)
        user_bias=user.loc[userId:userId, :].mean(axis=1).squeeze() - movie_avg
        print("ub",user_bias)

        movie = test.dropna(subset=[movieId])
        movie_bias=movie[movieId].mean()- movie_avg
      
        print("mb",movie_bias)
    
    
        baseline=movie_avg+movie_bias+user_bias
        print("baseline",baseline)

        test_user = test.loc[userId:userId, :]
        test_row = test_user - test_user.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)
        indices = nn.kneighbors(test_row, return_distance=False)[0]
      
        userIds = filtered_train.iloc[indices, :].index
      

      #if len(indices)<k:
       # myPreds.append(baseline)
      #else:
        sayac=0
        toplam2=0
        toplam_distance=0
        for i_user in userIds:
          user_rate=train.loc[i_user,movieId]
          print("user_rate",user_rate)
          user = train.dropna(subset=[i_user],axis=1)
          user_bias=user.loc[i_user:i_user, :].mean(axis=1).squeeze()- movie_avg
          print("ub_yakın",user_bias)

          movie = train.dropna(subset=[movieId])
          movie_bias=movie[movieId].mean()- movie_avg
          print("mb_yakın",movie_bias)
          baseline_user=movie_avg+movie_bias+user_bias
          print("baseline_user",baseline_user)
          distance = nn.kneighbors(test_row, return_distance=True)[0][0][sayac]
          print("distance",distance)
          toplam_distance +=distance 
          sayac+=1
          toplam2 += distance*(user_rate- baseline_user) 
          print("toplam2",toplam2)
  
        rating=baseline + (toplam2/ toplam_distance)
        print("rating",rating)
        myPreds.append(rating)
        real_label = U.loc[userId, movieId]
        realLabels.append(real_label)

In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.9828965590749917

# Prediction

In [ ]:
test = pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [ ]:
x=list(test.IDs.values)
userid=[]
movieId=[]

x=list(test.IDs.values)
for i in range(test.shape[0]):
  user,movie=x[i].split("_")
  userid.append(user)
  movieId.append(movie)


d = {'UserId':userid , 'MovieId': movieId}
df_Test = pd.DataFrame(data=d)

In [ ]:
d = {'UserId':userid , 'MovieId': movieId}
df_Test = pd.DataFrame(data=d)

In [ ]:
df_Test.head()

,UserId,MovieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697


In [ ]:
k = 50
myPreds = []

for i in range(len(df_Test)):
  userId=int(df_Test.UserId[i])
  movieId=int(df_Test.MovieId[i])
  filtered_train = U.dropna(subset=[movieId])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)
      
  if k>filtered_train.shape[0]:
    user = U.dropna(subset=[userId],axis=1)
    user_bias=user.loc[userId:userId, :].mean(axis=1).squeeze() - movie_avg
    movie = U.dropna(subset=[movieId])
    movie_bias=movie[movieId].mean()- movie_avg
    baseline=movie_avg+movie_bias+user_bias
    myPreds.append(baseline)
    
    
  else:
    nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
    nn.fit(filtered_train)

    user = U.dropna(subset=[userId],axis=1)
    user_bias=user.loc[userId:userId, :].mean(axis=1).squeeze() - movie_avg
    #print("ub",user_bias)

    movie = U.dropna(subset=[movieId])
    movie_bias=movie[movieId].mean()- movie_avg
  
    #print("mb",movie_bias)


    baseline=movie_avg+movie_bias+user_bias
    #print("baseline",baseline)

    test_user = U.loc[userId:userId, :]
    test_row = test_user - test_user.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)
    indices = nn.kneighbors(test_row, return_distance=False)[0]
  
    userIds = filtered_train.iloc[indices, :].index
  
    sayac=0
    toplam2=0
    toplam_distance=0
    for i_user in userIds:
      user_rate=U.loc[i_user,movieId]
      #print("user_rate",user_rate)
      user = U.dropna(subset=[i_user],axis=1)
      user_bias=user.loc[i_user:i_user, :].mean(axis=1).squeeze()- movie_avg
      #print("ub_yakın",user_bias)

      movie = U.dropna(subset=[movieId])
      movie_bias=movie[movieId].mean()- movie_avg
      #print("mb_yakın",movie_bias)
      baseline_user=movie_avg+movie_bias+user_bias
      #print("baseline_user",baseline_user)
      distance = nn.kneighbors(test_row, return_distance=True)[0][0][sayac]
      #print("distance",distance)
      toplam_distance +=distance 
      sayac+=1
      toplam2 += distance*(user_rate- baseline_user) 
      #print("toplam2",toplam2)

    rating=baseline + (toplam2/ toplam_distance)
    print("rating",rating)
    myPreds.append(rating)
      